In [1]:
import numpy as np 

# Cellular sheaves on graphs 
## Learning sheaf laplacian through minimum total variation approach 

In [2]:
# Let's generate a toy topology for our example

nodes = [i for i in range(7)]
edges = [
    (0,1),
    (0,2),
    (0,6),
    (1,3),
    (1,5),
    (2,3),
    (2,4),
    (3,4),
    (4,6),
    (5,6)
]

V = 7
E = len(edges)

In [3]:
d = 3                                           # Node and edges stalks dimension

F = {
    e:{
        e[0]:np.random.randn(3,3),
        e[1]:np.random.randn(3,3)
        } 
        for e in edges
    }                                           # Incidency linear maps

In [4]:
F

{(0,
  1): {0: array([[ 1.46233502,  0.41764618, -0.36519828],
         [ 0.20110168, -0.2495661 ,  1.65186211],
         [ 1.33133783, -0.13702468, -0.12818207]]), 1: array([[-1.03950529,  0.3567725 ,  0.52462796],
         [ 1.23787507,  0.24229884, -1.1860985 ],
         [-0.25999526,  0.65503442, -2.17859338]])},
 (0,
  2): {0: array([[-0.88096892, -1.65777365, -0.92802904],
         [-0.47890943,  0.79532257,  0.36444667],
         [ 2.04161637,  0.25417944,  0.24755026]]), 2: array([[-1.24679662, -0.12603921, -0.8345247 ],
         [-0.65846286, -0.7634452 ,  1.03108071],
         [ 0.24320028, -0.13817919, -1.45429805]])},
 (0,
  6): {0: array([[ 1.43576668,  0.44262854, -0.48715164],
         [ 0.04932993,  0.64450273,  0.02034057],
         [ 0.32113964,  0.14759831,  0.27880938]]), 6: array([[ 0.11437272,  0.62788655, -1.34422514],
         [ 1.03462007,  0.94999117, -0.71576269],
         [ 0.29144052, -0.79038696,  0.97647551]])},
 (1,
  3): {1: array([[ 1.31580876, -0.9927

In [5]:
# Graph representation

A = np.zeros((7,7))

for edge in edges:
    u = edge[0] 
    v = edge[1] 

    A[u,v] = 1
    A[v,u] = 1

D = np.diag(np.sum(A, axis = 0))
L = D - A

In [6]:
# Sheaf representation 

B = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = F[edge][u]
    B_v = F[edge][v]

    B[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

L_f = B.T @ B

In [7]:
B

array([[ 1.46233502,  0.41764618, -0.36519828,  1.03950529, -0.3567725 ,
        -0.52462796,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.20110168, -0.2495661 ,  1.65186211, -1.23787507, -0.24229884,
         1.1860985 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.33133783, -0.13702468, -0.12818207,  0.25999526, -0.65503442,
         2.17859338,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.88096892, -1.65777365, -0.92802904,  0.        

In [8]:
# Let's define a dataset through a diffusion process according to the sheaf laplacian 

N = 100
X = np.zeros((V*d,N))

In [9]:
X[0:3,:] = np.random.randn(3,N)

In [10]:
eigs, _ = np.linalg.eig(L_f)
eps = 2 / np.max(eigs)

In [11]:
X = ((np.eye(L_f.shape[0]) - eps*L_f)**3 @ X)

In [16]:
data = {node:X[node*d:(node+1)*d,:] for node in range(7)}

In [17]:
X_0 = data[0]
X_1 = data[1]

In [31]:
(X_1 @ X_0.T) @ np.linalg.inv(X_0 @ X_0.T)

array([[-2.24888977e+00, -7.65720722e-02,  2.47461414e-02],
       [ 1.59178105e+00,  5.38144038e-02,  1.73066630e-04],
       [-7.06601208e+00, -2.37982746e-01, -1.17475233e-02]])

In [ ]:
# Alternated Linear Maps Learning

def ALML(X_u, X_v, d, T = 50):
    # Initialization 

    F_u = np.eye(d)
    F_v = np.eye(d)

    # This matrices can be computed out of the learning loop 

    vu = X_v @ X_u.T
    uv = X_u @ X_v.T
    uu = np.linalg.inv(X_u @ X_u.T)
    vv = np.linalg.inv(X_v @ X_v.T)

    # Alternated learning through blockwise convex programs

    for t in range(T): 
        F_u = F_v @ vu @ uu
        F_v = F_u @ uv @ vv

    return F_u, F_v 